# Mise à jour du pandas-reader et importation librairie

In [ ]:
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importation des libraires

In [ ]:
import pandas as pd
import numpy as np 
import pandas_datareader.data as web #collecteur des données
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN,Embedding

# Récupération des données via le web

In [ ]:
data_yahoo = web.get_data_yahoo('^GSPC', start='2000-1-1',end='2022-08-31')

In [ ]:
data_yahoo.shape

(5703, 6)

In [ ]:
data_yahoo

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,1478.000000,1438.359985,1469.250000,1455.219971,931800000,1455.219971
2000-01-04,1455.219971,1397.430054,1455.219971,1399.420044,1009000000,1399.420044
2000-01-05,1413.270020,1377.680054,1399.420044,1402.109985,1085500000,1402.109985
2000-01-06,1411.900024,1392.099976,1402.109985,1403.449951,1092300000,1403.449951
2000-01-07,1441.469971,1400.729980,1403.449951,1441.469971,1225200000,1441.469971
...,...,...,...,...,...,...
2022-08-25,4200.540039,4147.589844,4153.259766,4199.120117,2976050000,4199.120117
2022-08-26,4203.040039,4057.659912,4198.740234,4057.659912,3175260000,4057.659912
2022-08-29,4062.989990,4017.419922,4034.580078,4030.610107,2963020000,4030.610107


In [ ]:
# récuperation des valeurs de la colonne Adj Close 
data = data_yahoo[["Adj Close"]].values
data

array([[1455.2199707 ],
       [1399.42004395],
       [1402.10998535],
       ...,
       [4030.61010742],
       [3986.15991211],
       [3955.        ]])

# Normalisation des données

In [ ]:
# Normalisation des valeurs récupérer
mms = MinMaxScaler()
data_normalisee = mms.fit_transform(data)
data_normalisee

array([[0.18900104],
       [0.17545746],
       [0.17611036],
       ...,
       [0.81409117],
       [0.80330237],
       [0.79573934]])

In [ ]:
print(data_normalisee[0:1,0] ) 
print(data_normalisee[1,0])

[0.18900104]
0.17545746256898181


# Création de X_train, y_train, X_test et y_test

In [ ]:
# Création de X_data et y_data
X_data = []
y_data = []
const=1
for i in range(len(data_normalisee)-1):
  X_data.append(data_normalisee[i:i+const,0])
  y_data.append(data_normalisee[i+const,0])

In [ ]:
# Transformation de X_data et y_data en array
X_data=np.array(X_data)
y_data=np.array(y_data)

X_data.shape,y_data.shape

((5702, 1), (5702,))

In [ ]:
# Redimensionnement de "X_data" en ajoutant une nouvelle dimension de shape "1"
X_data=np.reshape(X_data,(X_data.shape[0],X_data.shape[1],1))
X_data.shape

(5702, 1, 1)

In [ ]:
# Définition de X_train et y_train
long_train = int(len(X_data)*0.70)
X_train = X_data[0:long_train,:]
y_train = y_data[0:long_train]

In [ ]:
X_train.shape, y_train.shape

((3991, 1, 1), (3991,))

In [ ]:
# Définition de X_test et y_test
X_test = X_data[long_train:len(X_data),:]
y_test = y_data[long_train:len(y_data)]

In [ ]:
X_test.shape, y_test.shape

((1711, 1, 1), (1711,))

# Model RNN

In [ ]:
# Architecture RNN
def model_rnn(inputShape):
  model_rnn = Sequential()
  model_rnn.add(SimpleRNN(255,activation='relu',input_shape=inputShape,return_sequences=True))
  model_rnn.add(SimpleRNN(200,activation='relu',return_sequences=True))
  model_rnn.add(SimpleRNN(128,activation='relu',return_sequences=True))
  model_rnn.add(SimpleRNN(56,activation='relu',return_sequences=True))
  model_rnn.add(SimpleRNN(32,activation='relu',return_sequences=True))
  model_rnn.add(SimpleRNN(16,activation='relu',return_sequences=True))
  model_rnn.add(SimpleRNN(8,activation='relu',return_sequences=False))  
  model_rnn.add(Dense(1,activation='sigmoid'))
  return model_rnn

In [ ]:
model_rnn =model_rnn((X_train.shape[1],1))
model_rnn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_29 (SimpleRNN)   (None, 1, 255)            65535     
                                                                 
 simple_rnn_30 (SimpleRNN)   (None, 1, 200)            91200     
                                                                 
 simple_rnn_31 (SimpleRNN)   (None, 1, 128)            42112     
                                                                 
 simple_rnn_32 (SimpleRNN)   (None, 1, 56)             10360     
                                                                 
 simple_rnn_33 (SimpleRNN)   (None, 1, 32)             2848      
                                                                 
 simple_rnn_34 (SimpleRNN)   (None, 1, 16)             784       
                                                                 
 simple_rnn_35 (SimpleRNN)   (None, 8)                

In [ ]:
# Compilation du modèle RNN
model_rnn.compile(optimizer="adam",loss="mean_squared_error",metrics="RootMeanSquaredError")
model_rnn.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
125/125 [==============================] - 6s 8ms/step - loss: 1.6535e-05 - root_mean_squared_error: 0.0041
Epoch 2/100
125/125 [==============================] - 1s 9ms/step - loss: 1.4247e-05 - root_mean_squared_error: 0.0038
Epoch 3/100
125/125 [==============================] - 1s 9ms/step - loss: 1.4950e-05 - root_mean_squared_error: 0.0039
Epoch 4/100
125/125 [==============================] - 1s 9ms/step - loss: 1.4952e-05 - root_mean_squared_error: 0.0039
Epoch 5/100
125/125 [==============================] - 1s 9ms/step - loss: 1.4430e-05 - root_mean_squared_error: 0.0038
Epoch 6/100
125/125 [==============================] - 1s 9ms/step - loss: 1.4478e-05 - root_mean_squared_error: 0.0038
Epoch 7/100
125/125 [==============================] - 1s 9ms/step - loss: 1.5010e-05 - root_mean_squared_error: 0.0039
Epoch 8/100
125/125 [==============================] - 1s 9ms/step - loss: 1.5540e-05 - root_mean_squared_error: 0.0039
Epoch 9/100
125/125 [===================

In [ ]:
model_rnn.evaluate(X_test,y_test)

54/54 [==============================] - 1s 4ms/step - loss: 0.0888 - root_mean_squared_error: 0.2980


[0.08878833055496216, 0.29797372221946716]